In [ ]:
import os
import logging
from datetime import datetime
import json

import numpy as np
from csbdeep.models import Config
import onnxruntime as ort
import onnx

from flame import FLAMEImage, CAREInferenceSession
from flame.utils import min_max_norm
from flame.error import FLAMEImageError

2025-05-16 08:53:15.269744: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 08:53:15.280559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747410795.291554  523230 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747410795.294489  523230 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747410795.302380  523230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
INFERENCE_DATA_DIR = "/mnt/d/data/raw/0013_250514_HS6307_CAREtest_NA"
DATASET_DIRECTORY = "/mnt/d/code/Balu_CARE/datasets"
DATASET_NAME = "20250513_40I_denoising_7to40F"
DATASET_JSON = os.path.join(DATASET_DIRECTORY, f"{DATASET_NAME}.json")
MODEL_DIRECTORY = "/mnt/d/models/CARE/test_model"
MODEL_NAME = os.path.basename(MODEL_DIRECTORY)
ONNX_PATH = os.path.join(MODEL_DIRECTORY, f"{MODEL_NAME}.onnx")
JSON_PATH = os.path.join(MODEL_DIRECTORY, f"config.json")

for f in [DATASET_JSON, ONNX_PATH, JSON_PATH]:
    assert os.path.isfile(f)

### Loading Images

In [ ]:
images = []
for root, dirs, files in os.walk(INFERENCE_DATA_DIR):
    for f in files:
        if "tif" in f or "tiff" in f:
            try:
                this_image = FLAMEImage(
                    impath = os.path.join(root, f),
                    jsonext = "tileData.txt",
                    overrideNFrames = 1,
                    checkFrames = False,
                    checkZs = True
                )
            except FLAMEImageError as e:
                continue
            images.append(this_image)

### Loading ONNX

In [ ]:
session = CAREInferenceSession(
    model_path = ONNX_PATH,
    model_config_path = JSON_PATH,
    dataset_config_path = DATASET_JSON,
    inference_images = images,
    cpu_ok = False
)